In [2]:
# coding: utf-8
import pandas as pd
import jieba
import time
import paddle

In [3]:
#---------------------------------------中文分词---------------------------------
#添加使用者词典和停用词
paddle.enable_static()
jieba.enable_paddle()
jieba.load_userdict("../resource/dict.txt.big")              #自定义词典
stopwords = [line.strip() for line in open('../resource/stopwords.txt', 'r', encoding='utf-8').readlines()]

def txt_cut(juzi):
    return [w for w in jieba.lcut(juzi) if w not in stopwords]     #可增加len(w)>1

Paddle enabled successfully......
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/q9/0yjtkz2x1mq4f7qlvql797l80000gn/T/jieba.cache
Loading model cost 1.348 seconds.
Prefix dict has been built successfully.


In [4]:
#获取数据集
text = open('../1.txt',encoding='utf8').read()

In [5]:
#-------------------------------------情感词典读取-------------------------------
#注意：
#1.词典中怒的标记(NA)识别不出被当作空值,情感分类列中的NA都给替换成NAU
#2.大连理工词典中有情感分类的辅助标注(有NA),故把情感分类列改好再替换原词典中

# 扩展前的词典
df = pd.read_excel('../resource/情感词汇.xlsx')

df = df[['词语', '词性种类', '词义数', '词义序号', '情感分类', '强度', '极性']]

In [6]:
#-------------------------------------七种情绪的运用-------------------------------
Happy = []
Good = []
Surprise = []
Anger = []
Sad = []
Fear = []
Disgust = []

#df.iterrows()功能是迭代遍历每一行
for idx, row in df.iterrows():
    if row['情感分类'] in ['PA', 'PE']:
        Happy.append(row['词语'])
    if row['情感分类'] in ['PD', 'PH', 'PG', 'PB', 'PK']:
        Good.append(row['词语']) 
    if row['情感分类'] in ['PC']:
        Surprise.append(row['词语'])       
    if row['情感分类'] in ['NB', 'NJ', 'NH', 'PF']:
        Sad.append(row['词语'])
    if row['情感分类'] in ['NI', 'NC', 'NG']:
        Fear.append(row['词语'])
    if row['情感分类'] in ['NE', 'ND', 'NN', 'NK', 'NL']:
        Disgust.append(row['词语'])
    if row['情感分类'] in ['NAU']:     #修改: 原NA算出来没结果
        Anger.append(row['词语'])  

#正负计算不是很准 自己可以制定规则       
Positive = Happy + Good + Surprise
Negative = Anger + Sad + Fear + Disgust

In [7]:
#---------------------------------------情感计算---------------------------------
def emotion_caculate(text):
    
    anger = 0
    disgust = 0
    fear = 0
    sad = 0
    surprise = 0
    good = 0
    happy = 0

    anger_list = []
    disgust_list = []
    fear_list = []
    sad_list = []
    surprise_list = []
    good_list = []
    happy_list = []
    
    wordlist = txt_cut(text)
    #wordlist = jieba.lcut(text)
    wordset = set(wordlist)
    wordfreq = []
    for word in wordset:
        freq = wordlist.count(word)
        if word in Anger:
            anger+=freq
            anger_list.append(word)
        if word in Disgust:
            disgust+=freq
            disgust_list.append(word)
        if word in Fear:
            fear+=freq
            fear_list.append(word)
        if word in Sad:
            sad+=freq
            sad_list.append(word)
        if word in Surprise:
            surprise+=freq
            surprise_list.append(word)
        if word in Good:
            good+=freq
            good_list.append(word)
        if word in Happy:
            happy+=freq
            happy_list.append(word)
            
    emotion_info = {
        'length':len(wordlist),
        'anger': anger,
        'disgust': disgust,
        'fear':fear,
        'good':good,
        'sadness':sad,
        'surprise':surprise,
        'happy':happy,
        
    }

    indexs = ['length', 'anger', 'disgust','fear','sadness','surprise', 'good', 'happy']
    #return pd.Series(emotion_info, index=indexs), anger_list, disgust_list, fear_list, sad_list, surprise_list, good_list, happy_list
    return pd.Series(emotion_info, index=indexs)

In [8]:
res = emotion_caculate(text)
print(res)

length      4692
anger          0
disgust      139
fear          26
sadness       29
surprise       3
good         125
happy         35
dtype: int64


In [9]:
res_chart=pd.DataFrame(res)

In [10]:
from pyecharts import options as opts
from pyecharts.charts import Pie
x_data = ["disgust", "anger", "fear", "sadness", "surprise","good","happy"]
y_data = [int(res_chart.loc['anger']),int(res_chart.loc['disgust']) ,int(res_chart.loc['fear']) ,int(res_chart.loc['sadness']) ,int(res_chart.loc['surprise']),int(res_chart.loc['good']),int(res_chart.loc['happy']) ]
 
c = (
    Pie(init_opts=opts.InitOpts(width="1600px", height="1000px"))   # 图形的大小设置
    .add(
        series_name="访问来源",
        data_pair=[list(z) for z in zip(x_data, y_data)],
        radius=["15%", "50%"],   # 饼图内圈和外圈的大小比例
        center=["30%", "40%"],   # 饼图的位置：左边距和上边距
        label_opts=opts.LabelOpts(is_show=True),   # 显示数据和百分比  
    )
    .set_global_opts(legend_opts=opts.LegendOpts(pos_left="left", orient="vertical"))   # 图例在左边和垂直显示
    .set_series_opts(
        tooltip_opts=opts.TooltipOpts(
            trigger="item", formatter="{a} <br/>{b}: {c} ({d}%)"
        ),
    )
)
 
c.render_notebook()

In [11]:
c.render()

'/Users/z/Desktop/code/emotionClassification/render.html'